In [18]:
!pip install gradio --quiet
!pip install python-dotenv


## Deploy Diabetes Model on AWS SageMaker


In [20]:
import os
from dotenv import load_dotenv
import boto3
from sagemaker.sklearn.model import SKLearnModel
import time

# --- Load environment variables from the .env file ---
load_dotenv()

# --- Read values from the .env file ---
bucket_name = os.getenv("AWS_BUCKET_NAME")
s3_key = os.getenv("S3_MODEL_KEY")
local_path = os.getenv("LOCAL_MODEL_PATH")
role = os.getenv("SAGEMAKER_ROLE_ARN")
endpoint_name = os.getenv("SAGEMAKER_ENDPOINT_NAME")

# --- Upload model to S3 ---
s3 = boto3.client("s3")
s3.upload_file(local_path, bucket_name, s3_key)
print(f"Uploaded {local_path} to s3://{bucket_name}/{s3_key}")

# --- Initialize SageMaker client ---
sm = boto3.client("sagemaker")

# --- Delete old endpoint if exists ---
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
    print(f"Deleted existing endpoint: {endpoint_name}")
except sm.exceptions.ClientError:
    print("No existing endpoint to delete")

# --- Delete old endpoint config if exists ---
try:
    sm.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"Deleted existing endpoint config: {endpoint_name}")
except sm.exceptions.ClientError:
    print("No existing endpoint config to delete")

time.sleep(10)

# --- Define the SKLearn model ---
model_data = f"s3://{bucket_name}/{s3_key}"
sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    framework_version="1.2-1",
    entry_point="inference.py"
)

# --- Deploy the endpoint ---
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name=endpoint_name
)

print(f"Endpoint '{endpoint_name}' deployed successfully ")



Uploaded fixed_model/diabetes_model.tar.gz to s3://razan-mlops-demo/model/diabetes_model.tar.gz
Deleted existing endpoint: diabetes-endpoint
Deleted existing endpoint config: diabetes-endpoint
---------------!Endpoint 'diabetes-endpoint' deployed successfully 


## Test Deployed SageMaker Endpoint with Sample Input


In [10]:
import json
runtime = boto3.client("sagemaker-runtime")

data = {
    "age": 90,
    "bmi": 30.5,
    "glucose": 120,
    "blood_pressure": 80,
    "insulin": 90,
    "skin_thickness": 20,
    "pregnancies": 2,
    "diabetes_pedigree": 0.5,
    "feature9": 1.2,
    "feature10": 0.8
}

response = runtime.invoke_endpoint(
    EndpointName="diabetes-endpoint",
    ContentType="application/json",
    Body=json.dumps(data)
)

print("✅ Prediction:", response["Body"].read().decode())



✅ Prediction: {"prediction": [19754.37133022376]}


In [70]:
import joblib
import numpy as np

# Load the model
model = joblib.load("fixed_model/diabetes_model.pkl")

# Example input with all 10 features
sample_input = {
    "age": 90,
    "bmi": 30.5,
    "glucose": 120,
    "blood_pressure": 80,
    "insulin": 90,
    "skin_thickness": 20,
    "pregnancies": 2,
    "diabetes_pedigree": 0.5,
    "feature9": 1.2,
    "feature10": 0.8
}

FEATURE_ORDER = [
    "age", "bmi", "glucose", "blood_pressure", "insulin",
    "skin_thickness", "pregnancies", "diabetes_pedigree", "feature9", "feature10"
]

# Convert dict to ordered list and reshape to 2D
X = np.array([sample_input[feat] for feat in FEATURE_ORDER]).reshape(1, -1)

# Predict
print(model.predict(X))



[19754.37133022]


# Create a Gradio Web Interface for SageMaker Endpoint


In [9]:
import gradio as gr
import boto3
import json

runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")
ENDPOINT_NAME = "diabetes-endpoint"

def predict(age, bmi, glucose, blood_pressure, insulin, skin_thickness, pregnancies, diabetes_pedigree, feature9, feature10):
    data = {
        "age": age,
        "bmi": bmi,
        "glucose": glucose,
        "blood_pressure": blood_pressure,
        "insulin": insulin,
        "skin_thickness": skin_thickness,
        "pregnancies": pregnancies,
        "diabetes_pedigree": diabetes_pedigree,
        "feature9": feature9,
        "feature10": feature10
    }
    payload = json.dumps(data)
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Body=payload
    )
    result = json.loads(response["Body"].read().decode())
    return result["prediction"][0]

inputs = [
    gr.Number(label="Age"),
    gr.Number(label="BMI"),
    gr.Number(label="Glucose"),
    gr.Number(label="Blood Pressure"),
    gr.Number(label="Insulin"),
    gr.Number(label="Skin Thickness"),
    gr.Number(label="Pregnancies"),
    gr.Number(label="Diabetes Pedigree Function"),
    gr.Number(label="Feature 9"),
    gr.Number(label="Feature 10")
]

gr.Interface(fn=predict, inputs=inputs, outputs="number", title="Diabetes Prediction").launch(share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://95731b89b705c12441.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Delete Endpoint, Endpoint Config, and Model from S3 to Avoid Charges


In [13]:
sm = boto3.client("sagemaker")
sm.delete_endpoint(EndpointName="diabetes-endpoint")
sm.delete_endpoint_config(EndpointConfigName="diabetes-endpoint")


{'ResponseMetadata': {'RequestId': '59c24cab-c467-4015-8724-e7ec9dbb4b52',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '59c24cab-c467-4015-8724-e7ec9dbb4b52',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 12 Oct 2025 21:06:40 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [14]:
s3.delete_object(Bucket="razan-mlops-demo", Key="model/diabetes_model.tar.gz")


{'ResponseMetadata': {'RequestId': 'ZK19S65F7QYNKVH5',
  'HostId': 'Dj/h6pou+e8aKxXS8GbLUz+A6xWzz5l3a1sYhbCGJyW3I4HnX5ShsSoTt5WPo4+24R31NMewA1k=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Dj/h6pou+e8aKxXS8GbLUz+A6xWzz5l3a1sYhbCGJyW3I4HnX5ShsSoTt5WPo4+24R31NMewA1k=',
   'x-amz-request-id': 'ZK19S65F7QYNKVH5',
   'date': 'Sun, 12 Oct 2025 21:06:44 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}